# Data Preperation Microphone
Author: P.C.O. Maseland <br>
Date: 03 December 2021<br>

This ipynb file contains script to merge all seperate csv files into one whole dataset of Microphone.

In [1]:
#Import packages/libraries
import os
import pandas as pd
import csv
from zipfile import ZipFile
import numpy as np

#### Directory / file handling

In [2]:
#opening data files
path = 'Rawdata2/'
dirs = os.listdir(path)

In [3]:
#find folders
for file in dirs:
    print(file)

proband1
proband10
proband11
proband12
proband13
proband14
proband15
proband2
proband3
proband4
proband5
proband6
proband7
proband8
proband9


In [4]:
#create a list with all zip file paths
subpath = ''
count = 0
paths_list = []

for proband in dirs:
    subpath = path + proband
    subdir = os.listdir(subpath)
    for foldername in subdir:
        if foldername == 'data':
            subsubpath = subpath + "/data"
            subsubdir = os.listdir(subsubpath)
            for file in subsubdir:
                newpath = subsubpath + "/" + file
                paths_list.append(newpath)
                count += 1
        else:
            break

In [8]:
#all zip file paths
paths_list

['Rawdata2/proband1/data/acc_climbingdown_csv.zip',
 'Rawdata2/proband1/data/acc_climbingdown_sqlite.zip',
 'Rawdata2/proband1/data/acc_climbingup_csv.zip',
 'Rawdata2/proband1/data/acc_climbingup_sqlite.zip',
 'Rawdata2/proband1/data/acc_jumping_csv.zip',
 'Rawdata2/proband1/data/acc_jumping_sqlite.zip',
 'Rawdata2/proband1/data/acc_lying_csv.zip',
 'Rawdata2/proband1/data/acc_lying_sqlite.zip',
 'Rawdata2/proband1/data/acc_running_csv.zip',
 'Rawdata2/proband1/data/acc_running_sqlite.zip',
 'Rawdata2/proband1/data/acc_sitting_csv.zip',
 'Rawdata2/proband1/data/acc_sitting_sqlite.zip',
 'Rawdata2/proband1/data/acc_standing_csv.zip',
 'Rawdata2/proband1/data/acc_standing_sqlite.zip',
 'Rawdata2/proband1/data/acc_walking_csv.zip',
 'Rawdata2/proband1/data/acc_walking_sqlite.zip',
 'Rawdata2/proband1/data/gps_climbingdown_csv.zip',
 'Rawdata2/proband1/data/gps_climbingdown_sqlite.zip',
 'Rawdata2/proband1/data/gps_climbingup_csv.zip',
 'Rawdata2/proband1/data/gps_climbingup_sqlite.zip',


In [10]:
#remove sqlite.zip files and only use mic.zip files
paths_list_mic = [x for x in paths_list if "sqlite" not in x and "mic" in x]

#remove sqlite.zip files and only use light.zip files
paths_list_light = [x for x in paths_list if "sqlite" not in x and "lig" in x]

#### Zipfile handling
function 'openzip' has as input a zipfile path and SubjectNr. The function extract all csv files from zip, removes the readMe text file and retrieves info from file name:
* Sensory_type
* Activity
* Body-position

As output the function generates a list with csv file's and list of labels that stick with that csv file.

In [11]:
#input zipfilename, output list of csv files
def openzip(file_name, subject_id):
    labellist = []
    
    with ZipFile(file_name, 'r') as zip:
        csvlist = zip.namelist()
    
        if 'readMe' in csvlist:
            csvlist.remove('readMe')

        for name in csvlist:
            sublabellist = []
            zip.extract(name)

            #get labels
            templist = name.split("_")
            sensory_type = templist[0]
            activity = templist[1]
            body_position = templist[2].split(".")
            body_position = body_position[0]

            #create sublist in csvlist per csv file for labels
            sublabellist.append(sensory_type)
            sublabellist.append(activity)
            sublabellist.append(body_position)

            labellist.append(sublabellist)

        return csvlist, labellist

The 'Create_dataframe' function has as input the csvfile name, the according labellist and a count the nr of dataframe that is created. Since we merging per sensor-type, activity and subject the attributes. Meaning that we only need the Subject_ID, sensor-type and activty column 1 time. <br>

Further, the column attributes name will be changed according the body-position the data is from

With as output a dataframe.

In [16]:
def create_dataframe_mic(csvfile, labellist, count):
    df = pd.read_csv(csv)
    
    if count == 0:
        #add columns to df using labellist
        df['Subject ID'] = subject_ID
        df['Sensor-type'] = labellist[count][0]
        df['Activity'] = labellist[count][1]
          
    body_position = labellist[count][2]  
        
    df = df.rename(columns=
                {'attr_db': 'attr_db_' + body_position})
    return df

In [17]:
def create_dataframe_light(csvfile, labellist, count):
    df = pd.read_csv(csv)
    
    if count == 0:
        #add columns to df using labellist
        df['Subject ID'] = subject_ID
        df['Sensor-type'] = labellist[count][0]
        df['Activity'] = labellist[count][1]
          
    body_position = labellist[count][2]  
        
    df = df.rename(columns=
                {'attr_light': 'attr_light_' + body_position})
    return df

The main code path down below, retrieves for each zip file from which subject it is and generates a csv file containing the merged dataframe of each sensor-type, activity, subject combination. 

In [21]:
for path in paths_list_mic:
            #retrieve labels
            templist = path.split("/")
            templist2 = templist[3].split("_")

            #get subject label
            subject_ID = templist[1].replace("proband", '')

            print("PATH: " + path + " SUBJECT: " + subject_ID)

            count = 0
            csvlist, labellist = openzip(path, subject_ID)

            for csv in csvlist:

                if count == 0:
                    #first_df
                    df1 = create_dataframe_mic(csv, labellist, count)
                    #print('START dataframe    ' + csv)
                else:
                    #merge
                    df2 = create_dataframe_mic(csv, labellist, count)
                    merged_df = df1.merge(df2, on='id')
                    df1 = merged_df
                    #print('MERGE COMPLETE    ' + csv)

                count += 1

                #remove csv after use
                os.remove(csv)  

            #temporary csv
            filename = subject_ID + '_' + labellist[0][0] + '_' + labellist[0][1] + '_' + 'Merged.csv'
            print("MERGE FILE COMPLETE: " + filename)

            merged_df.to_csv(filename, index=True)

PATH: Rawdata2/proband1/data/mic_climbingdown_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_climbingdown_Merged.csv
PATH: Rawdata2/proband1/data/mic_climbingup_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_climbingup_Merged.csv
PATH: Rawdata2/proband1/data/mic_jumping_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_jumping_Merged.csv
PATH: Rawdata2/proband1/data/mic_lying_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_lying_Merged.csv
PATH: Rawdata2/proband1/data/mic_running_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_running_Merged.csv
PATH: Rawdata2/proband1/data/mic_sitting_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_sitting_Merged.csv
PATH: Rawdata2/proband1/data/mic_standing_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_standing_Merged.csv
PATH: Rawdata2/proband1/data/mic_walking_csv.zip SUBJECT: 1
MERGE FILE COMPLETE: 1_Microphone_walking_Merged.csv
PATH: Rawdata2/proband10/data/mic_climbingdown_csv.zip SUBJECT: 10
MERGE FILE COMP

MERGE FILE COMPLETE: 4_Microphone_climbingdown_Merged.csv
PATH: Rawdata2/proband4/data/mic_climbingup_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_climbingup_Merged.csv
PATH: Rawdata2/proband4/data/mic_jumping_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_jumping_Merged.csv
PATH: Rawdata2/proband4/data/mic_lying_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_lying_Merged.csv
PATH: Rawdata2/proband4/data/mic_running_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_running_Merged.csv
PATH: Rawdata2/proband4/data/mic_sitting_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_sitting_Merged.csv
PATH: Rawdata2/proband4/data/mic_standing_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_standing_Merged.csv
PATH: Rawdata2/proband4/data/mic_walking_csv.zip SUBJECT: 4
MERGE FILE COMPLETE: 4_Microphone_walking_Merged.csv
PATH: Rawdata2/proband5/data/mic_climbingdown_csv.zip SUBJECT: 5
MERGE FILE COMPLETE: 5_Microphone_climbingdown_Merged.csv
PATH: Rawdata2/proband5/

In [27]:
#adjust to what sensor you want a file to merge
#make dataset containing all csv's of accuracy
import glob
dir = os.getcwd()
dirs = os.listdir(dir)
sensor = 'Microphone'

for item in dirs:
    newdirs = glob.glob('*' + sensor + '*.csv') 
            

In [28]:
count = 0
for csv in newdirs:    
    if count == 0:
        df = pd.read_csv(csv)
    else: 
        df2 = pd.read_csv(csv)
        merged_df = df.append(df2)
        df = merged_df
        print(csv)
        print(merged_df.shape)
    count += 1

10_Microphone_climbingup_Merged.csv
(1217, 17)
10_Microphone_jumping_Merged.csv
(1354, 17)
10_Microphone_lying_Merged.csv
(2223, 17)
10_Microphone_running_Merged.csv
(3096, 17)
10_Microphone_sitting_Merged.csv
(3962, 17)
10_Microphone_standing_Merged.csv
(4855, 17)
10_Microphone_walking_Merged.csv
(5718, 17)
11_Microphone_climbingdown_Merged.csv
(6393, 17)
11_Microphone_climbingup_Merged.csv
(7244, 17)
11_Microphone_jumping_Merged.csv
(7375, 17)
11_Microphone_lying_Merged.csv
(8254, 17)
11_Microphone_running_Merged.csv
(9096, 17)
11_Microphone_sitting_Merged.csv
(9944, 17)
11_Microphone_standing_Merged.csv
(10793, 17)
11_Microphone_walking_Merged.csv
(11696, 17)
12_Microphone_climbingdown_Merged.csv
(12356, 17)
12_Microphone_climbingup_Merged.csv
(13115, 17)
12_Microphone_jumping_Merged.csv
(13251, 17)
12_Microphone_lying_Merged.csv
(13997, 17)
12_Microphone_running_Merged.csv
(14830, 17)
12_Microphone_sitting_Merged.csv
(15680, 17)
12_Microphone_standing_Merged.csv
(16516, 17)
12_Micr

In [29]:
merged_df.head(100)

,Unnamed: 0,id,attr_time_x,attr_db_chest,Subject ID,Sensor-type,Activity,attr_time_y,attr_db_head,attr_time_x.1,attr_db_shin,attr_time_y.1,attr_db_thigh,attr_time_x.2,attr_db_upperarm,attr_time_y.2,attr_db_waist,attr_time
0,0,1,1436802576474,15.064747,10,Microphone,climbingdown,1436802576043,29.889528,1436802576291,38.709475,1436802576152,31.903816,1.436803e+12,41.177399,1.436803e+12,28.885285,NaN
1,1,2,1436802577224,12.398478,10,Microphone,climbingdown,1436802576763,35.385425,1436802576998,31.604600,1436802576880,27.370913,1.436803e+12,40.271556,1.436803e+12,28.971935,NaN
2,2,3,1436802577982,3.596402,10,Microphone,climbingdown,1436802577504,49.765914,1436802577714,26.565574,1436802577620,22.587249,1.436803e+12,35.488124,1.436803e+12,35.296742,NaN
3,3,4,1436802578708,13.980829,10,Microphone,climbingdown,1436802578214,37.921671,1436802578452,28.382122,1436802578326,22.464601,1.436803e+12,27.171598,1.436803e+12,60.686873,NaN
4,4,5,1436802579369,33.656206,10,Microphone,climbingdown,1436802578870,37.320558,1436802579199,31.462733,1436802579065,23.009879,1.436803e+12,25.435472,1.436803e+12,54.175286,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,96,1436802648912,35.917948,10,Microphone,climbingdown,1436802644967,28.971055,1436802647329,70.278944,1436802645482,60.295938,1.436803e+12,33.331412,1.436803e+12,69.490338,NaN
96,96,97,1436802649634,40.726610,10,Microphone,climbingdown,1436802645682,73.773439,1436802648017,45.977630,1436802646177,63.325926,1.436803e+12,54.553376,1.436803e+12,60.595323,NaN
97,97,98,1436802650398,48.314906,10,Microphone,climbingdown,1436802646419,65.080899,1436802648671,70.418564,1436802646848,64.015377,1.436803e+12,56.659939,1.436803e+12,69.896534,NaN
98,98,99,1436802651154,58.807162,10,Microphone,climbingdown,1436802647142,62.810646,1436802649383,67.750540,1436802647600,67.543415,1.436803e+12,50.711381,1.436803e+12,65.434445,NaN


In [34]:
merged_df.columns

Index(['id', 'attr_db_chest', 'Subject ID', 'Sensor-type', 'Activity',
       'attr_db_head', 'attr_db_shin', 'attr_db_thigh', 'attr_db_upperarm',
       'attr_db_waist'],
      dtype='object')

In [31]:
#Drop unneccesary columns
merged_df = merged_df.drop(columns=['Unnamed: 0',
                     'attr_time',
                     'attr_time_x',
                     'attr_time_y',
                     'attr_time_x.1',
                     'attr_time_y.1',
                     'attr_time_x.2',
                     'attr_time_y.2'])

In [32]:
merged_df.to_csv("tempfile_mic.csv")

In [33]:
merged_df.shape

(85598, 10)

In [35]:
#look at NaN values
merged_df.isnull().sum(axis = 0)

id                     0
attr_db_chest          0
Subject ID             0
Sensor-type            0
Activity               0
attr_db_head           0
attr_db_shin           0
attr_db_thigh        126
attr_db_upperarm       0
attr_db_waist       1077
dtype: int64